In [1]:
# importing necessary modules
import cv2
import face_recognition
from imutils import paths
import pickle
import numpy as np

In [2]:
# declaring empty lists to store names and encodings later
known_names = []
known_encodings = []

# using the paths module to list out and find all images in the "Images" directory
image_paths = list(paths.list_images('Images'))
image_paths

In [3]:
# previewing the images 
image_paths[:20]

['Images\\Azeez\\azeez (1).jpg',
 'Images\\Azeez\\azeez (2).jpg',
 'Images\\Azeez\\azeez (3).jpg',
 'Images\\Azeez\\azeez (4).jpg',
 'Images\\Azeez\\azeez (5).jpg',
 'Images\\Azeez\\azeez (6).jpg',
 'Images\\Azeez\\azeez (7).jpg',
 'Images\\Azeez\\azeez (8).jpg',
 'Images\\Azeez\\azeez (9).jpg',
 'Images\\Biden\\biden1.jpeg',
 'Images\\Biden\\biden10.jpeg',
 'Images\\Biden\\biden11.jpeg',
 'Images\\Biden\\biden12.jpeg',
 'Images\\Biden\\biden13.jpeg',
 'Images\\Biden\\biden14.jpeg',
 'Images\\Biden\\biden15.jpeg',
 'Images\\Biden\\biden17.jpeg',
 'Images\\Biden\\biden18.jpeg',
 'Images\\Biden\\biden19.jpeg',
 'Images\\Biden\\biden2.jpeg',
 'Images\\Biden\\biden20.jpeg',
 'Images\\Biden\\biden21.jpeg',
 'Images\\Biden\\biden22.jpeg',
 'Images\\Biden\\biden3.jpeg',
 'Images\\Biden\\biden4.jpeg',
 'Images\\Biden\\biden6.jpeg',
 'Images\\Biden\\biden7.jpeg',
 'Images\\Biden\\biden8.jpeg',
 'Images\\Biden\\bidn9.jpeg',
 'Images\\Damola\\Damola (1).jpg',
 'Images\\Damola\\Damola (2).jpg',
 '

### Enrollment Stage

This is the "training" stage. The names and face encodings of the images are saved

In [4]:
# Looping through the images and saving the individual names
for image in image_paths:
    name = image.split('\\')[1]
    bgr_img = cv2.imread(image)
    
    # converting to RBG due to using the face_recognition library as cv2 reads images in BGR
    rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    
    # finding the locations of faces based on their facial features and encoding the faces 
    faces = face_recognition.face_locations(rgb_img, model='hog')
    encodings = face_recognition.face_encodings(rgb_img, faces)
    
    # saving the individual encodings and names to the pre-defined lists 
    for encoding in encodings:
        known_names.append(name)
        known_encodings.append(encoding)
        
# saving all encodings and names into a dictionary and serializing them into a pickled file 
data = {'names' : known_names, 'encodings' : known_encodings}
with open('face_encodings','wb') as f:
    f.write(pickle.dumps(data))

### Verification Stage

This is the stage where the serialized encodings are then deserialized and used to check new faces and flag them as recognized or not

In [ ]:
# reading the test image, resizing it and changing it to RBG
new_image = cv2.imread('Test Images/group (1).jpg')
new_image = cv2.resize(new_image, None, fx= 0.25, fy = 0.25)
rgb_new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)

# finding the face loactions and encodings of the test image (if any)
faces = face_recognition.face_locations(rgb_new_image)
encodings = face_recognition.face_encodings(rgb_new_image, faces)

# deserializing our encoding file 
with open('face_encodings', 'rb') as f:
    data = pickle.loads(f.read())

# creating empty structures
count_names = {}
names = []

# comparing the located faces in the test image to the serialized ones 
for encoding in encodings:
    
    # computing the distance between the serialized faces and the test face
    distances = face_recognition.face_distance(data['encodings'], encoding)
    min_value = np.min(distances)
    
    # setting a threshold value for the distance
    if min_value > 0.5:
        name = 'Unknown'
    else:
        i = np.argmin(distances)
        name = data['names'][i]
    names.append(name)

# drawing bounding boxes on all faces and labelling them 
face_names = list(zip(faces, names))
for((top, right, bottom, left), name) in face_names:
    cv2.rectangle(new_image, (left, top), (right, bottom), (255,0,0), 3)
    cv2.putText(new_image,name.upper(), (left-2,top-3), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
    
cv2.imshow('Picture', new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## END